# At Long Last, the AOE2 W-L Records!

This notebook displays how I have used Python code and AOE2 saved games to generate various summaries of W-L records.  Each saved game was renamed in a systematic way, and the file names are subsequently parsed and processed to produce these summaries.  File timestamps are also used here.  The raw dataset I have consists of all AOE2 (DE) saved games I have record of; therefore (for example), most of Ethan's ranked games will not be summarized here, since I simply don't have them; his "Ranked Game" W-L record as shown here is his W-L record in only the ranked games I have that he played in.

Here's a hypothetical example of a typical game filename, as renamed by myself after the game was initially recorded:

`Boh-Gur-Chi (me-Andy-Tom) v Azt-Hin-Per-Burg (Spencer-Ethan-Jared-Ryan) - arabia, got our trash kicked in, loss.aoe2record`

What this means is that I played in the game, with Andy and Tom as my teammates, against a team consisting of Spencer, Ethan, Jared, and Ryan.  Specifically, I played as the Bohemians ("Boh"), Andy as the Gurjaras ("Gur"), Tom as the Chinese ("Chi"), Spencer as the Aztecs ("Azt"), Ethan as the Hindustanis ("Hin"), Jared as the Persians ("Per"), and Ryan as the Burgundians ("Burg"), and the map we played was "Arabia."  The summary of the game is the text between the map specification and the result ("loss").  "Loss" means that my team lost the game and the other team won the game.

(Thankfully, this particular game was never actually played.)

This filename and the succeeding summary was similar to what I fed Codeium (an AI coder), which proceeded to give me a base program to start with.  However, there are, of course, other nuances to consider when parsing the various kinds of games—ranked, more than two teams, games played on game night as opposed to other times, etc.—which is why an endless number of conditionals were introduced and the code got messy really fast (you will please excuse my use of sub-optimal coding practices).

When quibbling about details of the W-L records you see, one thing to keep in mind is the fact that some games are marked "no decision," meaning none of the teams or players won or lost; this may be due to the kind of game it was, the situation in which the game was played, or simply because I wasn't sure whether or how to assign a winner.

As of today (5/9/25), there are a few features I'd like to include but haven't quite gotten to yet, including the ability to see W-L records for certain people as teammates or as opponents.  But there are nevertheless a bunch of things that still can be analyzed as it is; please follow along...

In [1]:
import aoe2_file_parser as aoe2
import random
from ipywidgets import interact
import pandas as pd

As can be seen, there's only the one Python file and one data file (again, not the best coding practices).  Comments kept toward the beginning of the Python file include the original instructions given to the AI coder as well as the list of \#TODOs I hope to eventually get to.

One glaring weakness is that I don't employ a database management system such as Neo4j, and this weakness will only become more evident as the (living) data document continues to increase with each new game played.  As this was more of an exercise and less of an attempt to optimize, and since it doesn't take too long for the parser to run through a few hundred lines of data, I decided not to bother for the time being, so this remains a \#TODO I eventually get to someday, hopefully.

In [2]:
filename = 'all_saved_games.xlsx'
games = aoe2.read_records_from_file(filename)

In [3]:
i = 0
record_dicts = []
for game in games:
    record_dicts.append(aoe2.parse_game_record(game))

To start off, let's see what my record playing as my favorite civ looks like.

In [4]:
brian_boh_games = aoe2.filter_player_games('Brian', civ='Boh', records=record_dicts)
summary = aoe2.summarize_records(brian_boh_games, "Brian's Games as Bohemians", min_games_played=len(brian_boh_games))

summary

[{'Player': 'Brian',
  "Brian's Games as Bohemians Record": '39-38',
  "Brian's Games as Bohemians Win %": '50.65'}]

OK, we could do this kind of thing for any player, and for any civ they've played.  I've created a list and a dictionary and set them as global constants:

GAME_NIGHT_PLAYERS: list of all familiar players who have joined us on game nights

CIV_DICT: dictionary of civilizations, keyed by their abbreviations (2-5 letters long)

In [5]:
# find each player's record for each civ.  use aoe2.summarize_records()
civ_records = {}
for player in aoe2.GAME_NIGHT_PLAYERS:
    civ_records[player] = []
    for civ in aoe2.CIV_DICT.keys():
        player_civ_games = aoe2.filter_player_games(player, civ=civ, records=record_dicts)
        summary = aoe2.summarize_records(player_civ_games, f"{player}'s {civ} Games", min_games_played=len(player_civ_games))
        if len(summary) == 0:
            continue
        for record in summary:
            if record['Player'] == player:
                civ_records[player].append({"Civ": f"{civ}", "Record": record[f"{player}'s {civ} Games Record"], "Win %": record[f"{player}'s {civ} Games Win %"]})

Let's see my record for all civs.

In [6]:
brian_df = pd.DataFrame(civ_records['Brian'])
brian_df

,Civ,Record,Win %
0,Arm,3-9,25.00
1,Azt,3-4,42.86
2,Ben,7-10,41.18
3,Ber,6-6,50.00
4,Boh,39-38,50.65
5,Brit,8-6,57.14
6,Bul,16-23,41.03
7,Burg,10-4,71.43
8,Burm,2-6,25.00
9,Byz,5-7,41.67


And a couple of others, for specific civs.

In [7]:
# Spencer's record as Italians
spencer_df = pd.DataFrame(civ_records['Spencer'])
spencer_df.loc[spencer_df['Civ'] == 'Ital']

,Civ,Record,Win %
22,Ital,8-3,72.73


In [8]:
# Ethan's record as Mongols
ethan_df = pd.DataFrame(civ_records['Ethan'])
ethan_df.loc[ethan_df['Civ'] == 'Mon']

,Civ,Record,Win %
23,Mon,17-6,73.91


Well how about the overall records already?!  Since this dataset includes a bunch of ranked games with and against a bunch of players I don't know, most of whom played in 1-3 games, we'd probably like to filter them out.  So I'll limit the summary to only those players who have played at least 10 games.  And lest you think I'm racist, I'm excluding players I've dubbed "[asian]" only because their handles included characters from Asian languages that I couldn't easily reproduce, and lumped them all into the same moniker.

This list includes the W-L records for our game night players in all games I have record of, including both ranked and unranked games.

In [9]:
# find W-L records of players who have played at least 10 games
overall_results = aoe2.summarize_records(record_dicts, "Overall", 10, ['AI', '[asian]'])
overall_results_df = pd.DataFrame(overall_results)
overall_results_df

,Player,Overall Record,Overall Win %
0,Spencer,147-95,60.74
1,Andy,184-151,54.93
2,Ethan,110-91,54.73
3,Ammon,31-26,54.39
4,Jared,78-80,49.37
5,MikeM,46-48,48.94
6,Tom,180-189,48.78
7,Joel,9-10,47.37
8,Brian,328-402,44.93
9,MikeS,95-118,44.60


Speaking of ranked and unranked games, let's set up some data structures so we can do some analysis with those.

In [10]:
ranked, unranked = [], []
for record in record_dicts:
    if record['ranked']:
        ranked.append(record)
    else:
        unranked.append(record)

# show what a random ranked game looks like
ranked[random.randint(0, len(ranked))]

{'epic': False,
 'game_night': False,
 'other_thursday': False,
 'ranked': True,
 'team_game': True,
 'map': 'fortress',
 'result': 'loss',
 'teams': {'team_1': {'Brian': 'Sic',
   'RarinWheel65511': 'Slob',
   'Diamondrob07634': 'Azt',
   'carloslicear': 'Per'},
  'team_2': {'mustymustard': 'Ital',
   'Luigi': 'Got',
   'Burgi53': 'Khm',
   'El Luchacabra': 'Teut'}}}

In [11]:
ranked_results = aoe2.summarize_records(ranked, "Ranked")
unranked_results = aoe2.summarize_records(unranked, "Unranked")

ranked_df = pd.DataFrame(ranked_results)
unranked_df = pd.DataFrame(unranked_results)

We can, of course, break it down further: say, looking at the unranked games that were played on our game night Thursdays.  And, for those of us who can't wait a full two weeks to play again, we have the "other" Thursday night games.

In [12]:
unranked_game_night_games = []
unranked_other_thursday_games = []
for game in unranked:
    if game['game_night']:
        unranked_game_night_games.append(game)
    elif game['other_thursday']:
        unranked_other_thursday_games.append(game)

unranked_game_night_summary = aoe2.summarize_records(unranked_game_night_games, "Game Night Unranked")
unranked_other_thursday_summary = aoe2.summarize_records(unranked_other_thursday_games, "Other Thursday Unranked")

unranked_game_night_summary_df = pd.DataFrame(unranked_game_night_summary)
unranked_other_thursday_summary_df = pd.DataFrame(unranked_other_thursday_summary)

Were you to scour the game records, you'd see the occasional "epic" tag.  These are games I marked thus that were the harder-fought struggles, that tended to be longer than your typical game, where everyone gave it their all and slogged through the slugfest.  Games we might actually like to watch the replays of; the kind of game you talk about over the water cooler and maybe even tell your grandchildren about.  These are the "epic" games, fully deserving of their appellations.

In [13]:
# find summary of epic games
unranked_epic_games = []
for game in unranked:
    if game['epic']:
        unranked_epic_games.append(game)

unranked_epic_summary = aoe2.summarize_records(unranked_epic_games, "Epic Unranked", min_games_played=5)
unranked_epic_summary_df = pd.DataFrame(unranked_epic_summary)

unranked_epic_summary_df

,Player,Epic Unranked Record,Epic Unranked Win %
0,Ammon,7-2,77.78
1,Spencer,24-11,68.57
2,Andy,20-12,62.50
3,Phelecia,3-2,60.00
4,MikeS,14-13,51.85
5,Jared,11-11,50.00
6,Ryan,12-15,44.44
7,MikeM,8-11,42.11
8,Tom,13-18,41.94
9,Brian,19-28,40.43


In [14]:
# find summary of ranked epic games
ranked_epic_games = []
for game in ranked:
    if game['epic']:
        ranked_epic_games.append(game)

ranked_epic_summary = aoe2.summarize_records(ranked_epic_games, "Epic Ranked", min_games_played=5)
ranked_epic_summary_df = pd.DataFrame(ranked_epic_summary)

ranked_epic_summary_df

,Player,Epic Ranked Record,Epic Ranked Win %
0,Andy,10-3,76.92
1,Tom,14-5,73.68
2,Spencer,5-3,62.50
3,Brian,22-15,59.46
4,MikeS,3-3,50.00
5,[asian],4-5,44.44


Well so I'd say it's about time we put everything together in a nice, neat, sortable summary, don't you think?

In [15]:
pd.options.display.expand_frame_repr = False

result_left = pd.merge(overall_results_df, unranked_game_night_summary_df, on='Player', how='left')
result_left = pd.merge(result_left, unranked_other_thursday_summary_df, on='Player', how='left')
result_left = pd.merge(result_left, unranked_df, on='Player', how='left')
result_left = pd.merge(result_left, ranked_df, on='Player', how='left')
result_left = pd.merge(result_left, unranked_epic_summary_df, on='Player', how='left')

result_left.set_index('Player', inplace=True)
result_left.sort_index(inplace=True)

In [16]:
def make_sortable(df):
    """Make a pandas DataFrame sortable by column in a Jupyter notebook."""

    def sort_and_display(by, ascending):
        df_sorted = df.sort_values(by=by, ascending=ascending)
        display(df_sorted)

    def combine_indices(columns, index):
        """Combine a DataFrame's columns with its index name."""
        return list(columns) + [index]

    win_pcts = [i for i in range(1, len(df.columns), 2)]
    indices = combine_indices(df.columns[win_pcts], df.index.name)
    
    widget = interact(sort_and_display, by=indices, ascending=False)
    display(widget)

In [17]:
make_sortable(result_left)

interactive(children=(Dropdown(description='by', options=('Overall Win %', 'Game Night Unranked Win %', 'Other…

<function __main__.make_sortable.<locals>.sort_and_display(by, ascending)>

You might notice that the "Game Night Unranked" and "Other Thursday Unranked" records don't add up to the overall "Unranked" records.  This is because there might have been the occasional game played between us game-night players on days other than Thursday.  For instance, my win % decreases drastically because of all my games played, with far too many losses, against Andy, my earliest nemesis.

Anyway, that's about it, folks!  As I said before, I'd like to include the ability to summarize records played with or against other individuals, or depending on the map played, etc., but haven't gotten to it yet.  Perhaps someday.  Or if someone would like to supply the ability themselves, we can add it to the repo...?